<a href="https://colab.research.google.com/github/learn-programmers/programmers_kdt_II/blob/main/9%EC%A3%BC%EC%B0%A8_PySpark_%EA%B8%B0%EB%B3%B8_2%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.(서버가 하나있는 개발용)

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2 MB 31 kB/s 
     |████████████████████████████████| 198 kB 17.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=01a53577218b654b7baed808dbc55099b5a0dee39c023c8887b63d760f1384e8
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [2]:
!ls -tl

total 4
drwxr-xr-x 1 root root 4096 Jul 16 13:20 sample_data


In [3]:
!ls -tl sample_data

total 55508
-rw-r--r-- 1 root root 18289443 Jul 16 13:20 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jul 16 13:20 mnist_train_small.csv
-rw-r--r-- 1 root root   301141 Jul 16 13:20 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jul 16 13:20 california_housing_train.csv
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md


**Spark Session:** SparkSession은 Spark 2.0부터 엔트리 포인트로 사용된다. 그 이전에는 SparkContext가 사용되었다. SparkSession을 이용해 RDD, 데이터 프레임등을 만든다. SparkSession은 SparkSession.builder를 호출하여 생성하며 다양한 함수들을 통해 세부 설정이 가능하다
* pandas와 달리 대용량, 병렬이 가능

In [4]:
from pyspark.sql import SparkSession

# master : 내가 지금 사용하고 싶은 spark cluster host name
# local : 로컬 spark cluster를 사용할 것이다.
# [*]  : 모든 cpu 사용 -> cpu의 숫자에 따라 partion이 결정 -> 병렬의 성능 결정
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

In [5]:
spark

**Python 객체를 RDD로 변환해보기**

**1> Python 리스트 생성**
* 데이터의 크기가 커지면 시간이 너무 느려짐

In [6]:
name_list_json = [ '{"name": "keeyong"}', '{"name": "benjamin"}', '{"name": "claire"}' ]

In [7]:
for n in name_list_json:
  print(n)

{"name": "keeyong"}
{"name": "benjamin"}
{"name": "claire"}


In [8]:
import json

for n in name_list_json:
  jn = json.loads(n)  # 파이썬 dictionary로 변환
  print(jn["name"])

keeyong
benjamin
claire


**2> 파이썬 리스트를 RDD로 변환. RDD로 변환되는 순간 Spark 클러스터의 서버들에 데이터가 나눠 저장됨 (파티션). 또한 Lazy Execution이 된다는 점 기억**

In [12]:
# sc = spark.sparkContext
# rdd = sc.parallelize(name_list_json)

rdd = spark.sparkContext.parallelize(name_list_json)  # rdd를 가지고 의미있는 코드를 실행하는 순간 만들어진다(lazy exeution)

In [13]:
rdd

ParallelCollectionRDD[2] at readRDDFromFile at PythonRDD.scala:262

In [14]:
rdd.count() 

3

In [18]:
# rdd에 새로운 명령어를 실행하여 새로운 rdd 객체를 만든다.
# 파이썬의 dictionary 타입으로 넣어진다.

parsed_rdd = rdd.map(lambda el:json.loads(el))  

In [19]:
parsed_rdd

PythonRDD[5] at RDD at PythonRDD.scala:53

In [20]:
parsed_rdd.collect()

[{'name': 'keeyong'}, {'name': 'benjamin'}, {'name': 'claire'}]

In [21]:
parsed_name_rdd = rdd.map(lambda el:json.loads(el)["name"])

In [23]:
# 로컬로 가져오는 collect()할 때 데이터가 너무 크면 오류 발생
# print가 안되고 다른 곳에 저장됨

parsed_name_rdd.collect()

['keeyong', 'benjamin', 'claire']

**파이썬 리스트를 데이터프레임으로 변환하기**

In [24]:
from pyspark.sql.types import StringType

df = spark.createDataFrame(name_list_json, StringType())

In [25]:
df.count()

3

In [26]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [27]:
df.select('*').collect()

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

In [28]:
df.select('value').collect()

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

In [29]:
from pyspark.sql import Row

row = Row("name") # Or some other column name
df_name = parsed_name_rdd.map(row).toDF()

In [30]:
df_name.printSchema()

root
 |-- name: string (nullable = true)



In [31]:
df_name.select('name').collect()

[Row(name='keeyong'), Row(name='benjamin'), Row(name='claire')]